In [ ]:
import numpy as np
import lime
import sklearn
import sklearn.ensemble
from anchor import utils
from anchor import anchor_tabular
import os
import sys

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)
from Anchor.anchor import Anchor, Tasktype

In [ ]:
data = np.genfromtxt('../datasets/titanic.txt', delimiter=',')
y_train = data[:, -1]
X_train = data[:, :-1]

c = sklearn.ensemble.RandomForestClassifier(n_estimators=100, n_jobs=5)
c.fit(X_train, y_train)
print('Train', sklearn.metrics.accuracy_score(y_train, c.predict(X_train)))

explainer = Anchor(Tasktype.TABULAR)

anchor = explainer.explain_instance(X_train[759].reshape(1, -1), c.predict, "smac", X_train, 100)

In [ ]:
anchor